In [18]:
from  __future__ import absolute_import
from __future__ import print_function
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from PIL import Image
import pickle
import shutil
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *
from DistanceLayer import DistanceLayer
from SiameseModel import SiameseModel
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)

# batch_size = 96
# batch_size = 8
# batch_size = 24
batch_size = 19
# batch_size = 3
# batch_size = 1

2.8.0


In [19]:
features = {}
# load the features dictionary from the file
with open('features_merged.pickle', 'rb') as handle:
# with open('features_mn.pickle', 'rb') as handle:
    features = pickle.load(handle)

feature_dim = features["02461"].shape[0]

# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [20]:
embedding = tf.keras.Sequential([
    Input(shape=(feature_dim,)),
    # tf.keras.layers.BatchNormalization(),
    Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    Dropout(0.6),
    Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    Dropout(0.6),
    Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

anchor_input = layers.Input(name="anchor", shape=(feature_dim))
positive_input = layers.Input(name="positive", shape=(feature_dim))
negative_input = layers.Input(name="negative", shape=(feature_dim))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [21]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*feature_dim,
    )

table.insert(keys, values)

In [22]:
train_triplets = np.loadtxt('./train_triplets_split.txt', dtype=str,delimiter = ' ')
val_triplets = np.loadtxt('./val_triplets_split.txt', dtype=str,delimiter = ' ')
train_dataset = tf.data.Dataset.from_tensor_slices(train_triplets)
val_dataset = tf.data.Dataset.from_tensor_slices(val_triplets)
count = np.array(train_triplets).shape[0]
print(count)


@tf.function
def load_image(inputs):
    filenames = inputs
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]
    output = (anchor,positive,negative)
    
    return output

train_dataset = train_dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)
val_dataset = val_dataset.map(load_image).cache().batch(batch_size).prefetch(20)

7315


In [23]:
siamese_model = SiameseModel(model,margin=0.5)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


In [24]:

#this is needed, otherwise nothing works
pred = siamese_model.predict(next(iter(train_dataset)))
# print(pred)

In [25]:
train_steps_per_epoch = int(count/batch_size)
checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
#save checkpoint after every epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
)


siamese_model.fit(train_dataset, epochs=100,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback],validation_data=val_dataset)



Epoch 1/100
385/385 [==============================] - 18s 45ms/step - loss: 0.4380 - val_accuracy: 0.7187
Epoch 2/100
385/385 [==============================] - 18s 48ms/step - loss: 0.3543 - val_accuracy: 0.7372
Epoch 3/100
385/385 [==============================] - 19s 49ms/step - loss: 0.3083 - val_accuracy: 0.7451
Epoch 4/100
384/385 [============================>.] - ETA: 0s - loss: 0.2896

In [ ]:
# tf.keras.utils.plot_model(
#     model,
#     to_file="model.png",
#     show_shapes=True,
#     expand_nested=True,
    
# )